In [561]:
# Import Dependencies
import pymongo
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
from splinter import Browser

In [562]:
# Find path to our chrome browser

# !which chromedriver

In [581]:
# Set path to browser & start splinter

# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome')

In [582]:
# Open webpage

url = 'http://quotes.toscrape.com/'
browser.visit(url)

In [583]:
# Create BS object

html = browser.html
soup_obj = soup(html, 'html.parser')

In [584]:
results = soup_obj.find_all("body")
results

[<body>
 <div class="container">
 <div class="row header-box">
 <div class="col-md-8">
 <h1>
 <a href="/" style="text-decoration: none">Quotes to Scrape</a>
 </h1>
 </div>
 <div class="col-md-4">
 <p>
 <a href="/login">Login</a>
 </p>
 </div>
 </div>
 <div class="row">
 <div class="col-md-8">
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" hr

In [585]:
quote_block = soup_obj.findAll("div", class_="quote")
# # for quote in quote_block:

# # # # # # Successfully finds each quote, tage and author on the page (except for tag which just lists all of them instead of group them per quote)
# quote_text = soup_obj.findAll("span", class_ = "text")
# quote_tags = soup_obj.findAll("a", class_ = "tag")
# quote_author = soup_obj.findAll("small", class_="author")

# # # # # # Lists for each for loop of texts, tags, and author
# text_list = []
# tags_list = []
# author_list = []

# # # # # # For loop for each to put them in the list above
# for text in quote_text:
#     text_list.append(text.text)
# #     print(text.text)

# for tags in quote_tags:
#     tags_list.append(tags.text)
# #     print(tags.text)

# for author in quote_author:
#     author_list.append(author.text)
# #     print(author.text)

# # # # # After browser click (will change when looping between pages)
# author_date = soup_obj.find('span', class_="author-born-date").text
# author_location = soup_obj.find('span', class_="author-born-location").text
# author_descrp = soup_obj.find('div', class_="author-description").text
# final_decrp = author_descrp.split('\n')[1].strip()

# author_date_list = []
# author_location_list = []
# final_decrp_list = []



In [586]:
# Fail at looping

# all_data = []
# for quote in quote_block:
#     quote_text = soup_obj.findAll("span", class_ = "text")
#     quote_tags = soup_obj.findAll("a", class_ = "tag")
#     quote_author = soup_obj.findAll("small", class_="author")
#     data = {
#         'text': quote_text,
#         'tags': quote_tags,
#         'author': quote_author
#     }
#     all_data.append(data)
# all_data

In [587]:
# # # Loop concept for going through each page and then clicking on about to get each author's info
# for loop "page in page_end" collect list of each item 
#     for loop x in total **find_author = browser.find_by_text('(about)')**
#         find_author.click()
#         find author date, location, and decrp
#         click back to previous page

all_data = []
for x in results:
    for quote in quote_block:
        all_data.append(quote.text.strip().split('\n'))
#         print(quote.text)

# List of all quotes, author, and tags in its own list
# all_data[0][0]
all_data

[['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'by Albert Einstein',
  '(about)',
  '',
  '',
  '            Tags:',
  '            ',
  'change',
  'deep-thoughts',
  'thinking',
  'world'],
 ['“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'by J.K. Rowling',
  '(about)',
  '',
  '',
  '            Tags:',
  '            ',
  'abilities',
  'choices'],
 ['“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'by Albert Einstein',
  '(about)',
  '',
  '',
  '            Tags:',
  '            ',
  'inspirational',
  'life',
  'live',
  'miracle',
  'miracles'],
 ['“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'by Jane Austen',
  '(about)',
  '',
  '',
  '            Tags:',
  '            ',
  'aliteracy',
  'books',
  'classi

In [588]:
# The loops works but theres one problem with page 3 where it doesn't
# retrieve the author information (descrp, location, and date) so I had
# to write a try function to work around that and add something in its
# place. Even though the loop works the tags are still how they were 
# before. One thing we'll need to fix is how to stop the loop when we get
# to the end of the pages. The only way I could do it is if I tell it to 
# stop at 10.

text_list = []
tags_list = []
author_list = []
author_date_list = []
author_location_list = []
final_decrp_list = []
count = 0
while True:
    count += 1
    for stuff in results:
        
        html = browser.html
        soup_obj = soup(html, 'html.parser')
        
        quote_block = soup_obj.findAll("div", class_="quote")

#         Finds quotes
        quote_text = soup_obj.findAll("span", class_ = "text")
        for text in quote_text:
            text_list.append(text.text) 
            
#         Finds tags
        quote_tags = soup_obj.findAll("a", class_ = "tag")
        for tags in quote_tags:
            tags_list.append(tags.text) 
            
#         Finds author
        quote_author = soup_obj.findAll("small", class_="author")
        for author in quote_author:
            author_list.append(author.text)


# # # # # # This is an href list but I had to get rid of all the tags I knew wouldn't be needed.
        links_list = []
        links = soup_obj.find_all('a')
        for link in links:
            links_list.append(link['href'])

        links_list.pop(0) # Pesky forward slash that wasn't needed
        links_list = [x for x in links_list if not 'tag' in x] # Got rid of tag links
        links_list = [x for x in links_list if not 'page' in x] # Got rid of page links
        links_list = [x for x in links_list if not 'login' in x] # Got rid of login link
        links_list = [x for x in links_list if not 'https' in x] # Got rid of https links


        for click in links_list:
            try:
                browser.click_link_by_href(click)
            except Exception:
                pass 
            
            html = browser.html
            soup_obj = soup(html, 'html.parser')    

            author_date = soup_obj.find('span', class_="author-born-date")
            author_location = soup_obj.find('span', class_="author-born-location")
            author_descrp = soup_obj.find('div', class_="author-description")
            try:
                author_date_list.append(author_date.text)
                author_location_list.append(author_location.text)
                final_decrp_list.append(author_descrp.text)
                browser.back()
#             except Exception:
#                 pass
            except:
                author_date_list.append('*****Could Not Retrieve*****')
                author_location_list.append('*****Could Not Retrieve*****')
                final_decrp_list.append('*****Could Not Retrieve*****')  
            


# # # # # # Next Page 
        html = browser.html
        soup_obj = soup(html, 'html.parser')
        page_link = []
        all_links = soup_obj.find_all('a')
        for p_link in all_links:
            page_link.append(p_link['href'])
        page_link = [x for x in page_link if not 'tag' in x]
        page_link = [x for x in page_link if 'page' in x]
        try:
            browser.click_link_by_href(page_link[1])
        except:
            browser.click_link_by_href(page_link[0])

    if count == 10:
        break
    print(count)

1
2
3
4
5
6
7
8
9


In [589]:
# # # # # # Used this just to help form the links list for the above loop

html = browser.html
soup_obj = soup(html, 'html.parser')

links_list = []
links = soup_obj.find_all('a')
for link in links:
    links_list.append(link['href'])
# # # # # # Used this just to form the links list for the above loop
links_list.pop(0) # Pesky forward slash that wasn't needed
links_list = [x for x in links_list if not 'tag' in x] # Got rid of tag links
links_list = [x for x in links_list if not 'page' in x] # Got rid of page links
links_list = [x for x in links_list if not 'login' in x] # Got rid of login link
links_list = [x for x in links_list if not 'https' in x] # Got rid of https links
links_list

['/author/Albert-Einstein',
 '/author/Jane-Austen',
 '/author/J-K-Rowling',
 '/author/Jane-Austen',
 '/author/Jane-Austen',
 '/author/C-S-Lewis',
 '/author/C-S-Lewis',
 '/author/Mark-Twain',
 '/author/Mark-Twain',
 '/author/C-S-Lewis']

In [598]:
# # # # # # Made sure lists were being filled

# text_list
# tags_list
# author_list
# author_date_list
# author_location_list
# final_decrp_list

author_list
# This was the last thing I did. The rest is unchanged from
# when we did it in class.

['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin',
 'Marilyn Monroe',
 'J.K. Rowling',
 'Albert Einstein',
 'Bob Marley',
 'Dr. Seuss',
 'Douglas Adams',
 'Elie Wiesel',
 'Friedrich Nietzsche',
 'Mark Twain',
 'Allen Saunders',
 'Pablo Neruda',
 'Ralph Waldo Emerson',
 'Mother Teresa',
 'Garrison Keillor',
 'Jim Henson',
 'Dr. Seuss',
 'Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Bob Marley',
 'Dr. Seuss',
 'J.K. Rowling',
 'Bob Marley',
 'Mother Teresa',
 'J.K. Rowling',
 'Charles M. Schulz',
 'William Nicholson',
 'Albert Einstein',
 'Jorge Luis Borges',
 'George Eliot',
 'George R.R. Martin',
 'C.S. Lewis',
 'Marilyn Monroe',
 'Marilyn Monroe',
 'Albert Einstein',
 'Marilyn Monroe',
 'Marilyn Monroe',
 'Martin Luther King Jr.',
 'J.K. Rowling',
 'James Baldwin',
 'Jane Austen',
 'Eleanor Roosevelt',
 'Marilyn Monroe',
 'Albert Einstein'

In [13]:
# Complete For Loop [NEED TO FIGURE THIS OUT]

# quote_block = soup_obj.findAll("div", class_="quote")
# for quote in quote_block:
#     quote_text = soup_obj.find("span", class_="text").text
#     print(quote_text)
#     quote_tags = soup_obj.findAll("a", class_="tag").text
#     print(quote_tags)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“The world as we have created it is a process of our thinking. It cannot

In [71]:
# Find first quote text

first_quote = soup_obj.find("span", class_="text").text
first_quote

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [155]:
# Find first quote tags [NEED TO FIGURE THIS OUT]

# first_block = soup_obj.find('div', class_="quote")
# for tag in first_block:
#     tags = tag.find('a', class_="tag").text
#     print(tags)

# tag_text = []
# first_block = soup_obj.find('div', class_="quote")
# for tag in first_block:
#     tags = soup_obj.find('a', class_="tag").text
#     tag_text.append(tags)
# tag_text

# first_quote_tags = soup_obj.find("a", class_="tag").text
# first_quote_tags

# tag_text = []
# for tag in soup_obj.find('div', class_="quote"):
#     tags = soup_obj.a
#     tags.next_sibling
# print(tags)

first_quote_tags = soup_obj.findAll("a", class_="tag")
first_quote_tags

[<a class="tag" href="/tag/change/page/1/">change</a>,
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>,
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>,
 <a class="tag" href="/tag/world/page/1/">world</a>,
 <a class="tag" href="/tag/abilities/page/1/">abilities</a>,
 <a class="tag" href="/tag/choices/page/1/">choices</a>,
 <a class="tag" href="/tag/inspirational/page/1/">inspirational</a>,
 <a class="tag" href="/tag/life/page/1/">life</a>,
 <a class="tag" href="/tag/live/page/1/">live</a>,
 <a class="tag" href="/tag/miracle/page/1/">miracle</a>,
 <a class="tag" href="/tag/miracles/page/1/">miracles</a>,
 <a class="tag" href="/tag/aliteracy/page/1/">aliteracy</a>,
 <a class="tag" href="/tag/books/page/1/">books</a>,
 <a class="tag" href="/tag/classic/page/1/">classic</a>,
 <a class="tag" href="/tag/humor/page/1/">humor</a>,
 <a class="tag" href="/tag/be-yourself/page/1/">be-yourself</a>,
 <a class="tag" href="/tag/inspirational/page/1/">inspirational</a>,
 

In [57]:
# Find first quote author name

first_quote_author = soup_obj.find("small", class_="author").text
first_quote_author

'Albert Einstein'

In [233]:
# Find first quote author born & description

find_author = browser.find_by_text('(about)')
find_author.click()

In [85]:
author_date = soup_obj.find('span', class_="author-born-date").text
author_location = soup_obj.find('span', class_="author-born-location").text
author_date, author_location

('March 14, 1879', 'in Ulm, Germany')

In [97]:
author_descrp = soup_obj.find('div', class_="author-description").text
final_decrp = author_descrp.split('\n')[1].strip()
final_decrp

'In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for the ethical good, teachers of religion must have the stature to give up the doctrine of a personal God, that is, give up that source of fear and hope which in the past placed such vast power in the hands of priests. In their labors they will have to avail themselves of those forces which are c